## 기본 Import

바로 아래 셀은 한국어 실행을 위한 셀입니다. <br>
실행 뒤에 런타임 재시작 후, 다음 셀부터 시작하시면 됩니다.

In [ ]:
!pip install matplotlib -U
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['font.family']=['NanumGothic','sans-serif']
plt.rcParams['axes.unicode_minus']=False
import numpy as np
import pandas as pd
import re
import seaborn as sns
import matplotlib as mpl
import platform
import warnings
import datetime
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.options.display.max_rows = 5500
# pd.options.display.max_columns = 20

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#구글드라이브연동

Mounted at /content/drive


In [ ]:
# !pip install sweetviz
!pip install pytimekr

  Preparing metadata (setup.py) ... done
  Created wheel for pytimekr: filename=pytimekr-0.1.0-py3-none-any.whl size=7922 sha256=c5d5e42f85d05b8fa193449a8a9473e621f969afe090b1f3fd34cf8c689a87d9
  Stored in directory: /root/.cache/pip/wheels/b3/a3/ba/e73aaae1dec550b8667510c379b019295bba855275258745de
Successfully built pytimekr


In [ ]:
from pytimekr import pytimekr

## Data 로드

In [ ]:
content = pd.read_excel('/content/drive/MyDrive/hv_project/Data/contents.xlsx', engine='openpyxl')
vod = pd.read_excel('/content/drive/MyDrive/hv_project/Data/vod.xlsx', engine='openpyxl')
channel = pd.read_csv('/content/drive/MyDrive/hv_project/Data/channel.csv', encoding="cp949", header=None)

In [ ]:
content.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13635 entries, 0 to 13634
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   subsr           13635 non-null  int64 
 1   series_nm       6451 non-null   object
 2   super_asset_nm  13635 non-null  object
 3   ct_cl           13635 non-null  object
 4   genre_of_ct_cl  13635 non-null  object
 5   log_dt          13635 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 639.3+ KB


In [ ]:
vod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5042 entries, 0 to 5041
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   subsr           5042 non-null   int64 
 1   asset_nm        5042 non-null   object
 2   ct_cl           5042 non-null   object
 3   genre_of_ct_cl  5042 non-null   object
 4   use_tms         5042 non-null   int64 
 5   strt_dt         5042 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 236.5+ KB


In [ ]:
channel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 989942 entries, 0 to 989941
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   0       989942 non-null  int64 
 1   1       989942 non-null  object
 2   2       989942 non-null  int64 
 3   3       989942 non-null  int64 
 4   4       989942 non-null  object
dtypes: int64(3), object(2)
memory usage: 37.8+ MB


## 데이터 탐색

### null 탐색

데이터 프레임에서 null인 친구들이 있을까?

In [ ]:
import pandas as pd
columns_with_null = content.columns[content.isnull().any()]
print('contents')
for col in columns_with_null:
    print(f"Column '{col}' has null/NaN values.")
print('vod')
columns_with_null = vod.columns[vod.isnull().any()]
for col in columns_with_null:
    print(f"Column '{col}' has null/NaN values.")
print('channel')
columns_with_null = channel.columns[channel.isnull().any()]
for col in columns_with_null:
    print(f"Column '{col}' has null/NaN values.")

contents
Column 'series_nm' has null/NaN values.
vod
channel


### Content

In [ ]:
content.head()

,subsr,series_nm,super_asset_nm,ct_cl,genre_of_ct_cl,log_dt
0,66885000,나는자연인이다,나는자연인이다 516회,TV 시사/교양,인물/다큐,20230917085318
1,66885000,나는자연인이다,나는자연인이다 516회,TV 시사/교양,인물/다큐,20230920161048
2,64385000,MBC 네트워크 특선,MBC 네트워크 특선 845회,TV 시사/교양,기타,20230922190616
3,64659000,이슈 픽 쌤과 함께,이슈 픽 쌤과 함께 145회,TV 시사/교양,기타,20230908153822
4,64659000,이슈 픽 쌤과 함께,이슈 픽 쌤과 함께 145회,TV 시사/교양,기타,20230908144512


In [ ]:
# content[content['genre_of_ct_cl']=='드라마']
unique_genres = content[content['ct_cl'] == 'TV드라마']['series_nm'].unique()
len(unique_genres)
# unique_genres

536

In [ ]:
vod.head()

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,strt_dt,super_asset_nm
0,64659000,과학수사대 스모킹 건 18회(23/08/02),TV 시사/교양,기타,3120,20230902121124,과학수사대 스모킹 건
1,64536000,(HD)그것이알고싶다 1364회(23/08/12),TV 시사/교양,기타,4260,20230930102607,(HD)그것이알고싶다
2,66873000,(HD)그것이알고싶다 1366회(23/08/26),TV 시사/교양,기타,4388,20230930042158,(HD)그것이알고싶다
3,62421000,과학수사대 스모킹 건 13회(23/06/21),TV 시사/교양,기타,3180,20230902123420,과학수사대 스모킹 건
4,66307000,식객 허영만의 백반기행 206회(23/07/07),TV 시사/교양,인물/다큐,244,20230902192639,식객 허영만의 백반기행


In [ ]:
unique_genres = vod[vod['genre_of_ct_cl'] == '외화']['super_asset_nm'].unique()
unique_genres

array([], dtype=object)

subsr의 unique 개수

In [ ]:
len(content['subsr'].unique())

648

content의 컬럼을 구경해보자

In [ ]:
print('ct_cl의 종류', content['ct_cl'].unique())
print()
print('ct_cl의 장르 종류',content['genre_of_ct_cl'].unique())
print()
print('series_nm의 종류 개수',len(content['series_nm'].unique()))
print()
print('super_asset_nm(회차 포함)의 종류 개수',len(content['super_asset_nm'].unique()))
print()
print('content row', len(content))

ct_cl의 종류 ['TV 시사/교양' 'TV 연예/오락' 'TV드라마' 'TV애니메이션' '공연/음악' '다큐' '라이프' '미분류' '스포츠'
 '영화' '우리동네' '키즈']

ct_cl의 장르 종류 ['인물/다큐' '기타' '외화 시리즈' '명랑/코믹' '액션/모험' '학원/순정/연애' '무협/환타지' '추리/미스터리' '스포츠'
 '인물' '자연' '역사' '운동/건강' '미분류' '레슬링' 'SF/환타지' '공포/스릴러' '다큐멘터리' '단편' '드라마'
 '로맨틱코미디' '멜로' '무협' '애니메이션' '액션/어드벤쳐' '에로틱' '코미디' '연예/오락' '뉴스' '학습' '오락']

series_nm의 종류 개수 1031

super_asset_nm(회차 포함)의 종류 개수 3885

content row 13635



 0   subsr           13821 non-null  int64  셋톱박스 ID 로 추정됩니다.<br>
 1   series_nm       6637 non-null   object 시리즈 컨텐츠의 이름<br>
 2   super_asset_nm  13821 non-null  object 컨텐츠의 이름 <br>
 3   ct_cl           13821 non-null  object 컨텐츠 자체 종류 <br>
 4   genre_of_ct_cl  13821 non-null  object 종류의 장르 <br>
 5   log_dt          13821 non-null  int64  해당 컨텐츠에 접근한 시간?

In [ ]:
content['log_dt'] = content['log_dt'].apply(lambda x:pd.to_datetime(str(x),format="%Y%m%d%H%M%S"))

In [ ]:
content2=content

In [ ]:
import re
li = content2['super_asset_nm'].unique()
unique_contents = set()
for i in li:
    if isinstance(i, str):
        matches = re.findall(r'\((.*?)\)', i)
        for match in matches:
            unique_contents.add(f'({match})')
for content in unique_contents:
    print(content)

In [ ]:
# 'super_asset_nm' 값 중에서 공백 문자열로 시작하는 경우 찾기
condition = content['super_asset_nm'].str.startswith(' ')
rows_with_condition = content[condition]
# 조건을 만족하는 'super_asset_nm' 값 출력
print(rows_with_condition['super_asset_nm'])

ValueError: ignored

In [ ]:
content['super_asset_nm'] = content['super_asset_nm'].astype(str)
content['series_nm'] = content.apply(lambda row: row['super_asset_nm'] if '회' not in row['super_asset_nm'] else re.sub(r'\d+\s*회.*', '', row['super_asset_nm']), axis=1)

In [ ]:
def is_korean_holiday(date):
    # 주말 여부 확인 (토요일: 5, 일요일: 6)
    if date.weekday() >= 5:
        return 1  # 주말 (토요일 또는 일요일)

    # pytimekr를 사용하여 모든 연도의 공휴일 정보를 가져옴
    holidays = pytimekr.holidays()
    korean_holidays = set()
    for holiday_date in holidays:
        korean_holidays.add(holiday_date)
    if date in korean_holidays:
        return 1  # 공휴일
    else:
        return 0  # 평일
content['is_holiday'] = content['log_dt'].apply(is_korean_holiday)

In [ ]:
content.head()

,subsr,series_nm,super_asset_nm,ct_cl,genre_of_ct_cl,log_dt
0,66885000,나는자연인이다,나는자연인이다 516회,TV 시사/교양,인물/다큐,2023-09-17 08:53:18
1,66885000,나는자연인이다,나는자연인이다 516회,TV 시사/교양,인물/다큐,2023-09-20 16:10:48
2,64385000,MBC 네트워크 특선,MBC 네트워크 특선 845회,TV 시사/교양,기타,2023-09-22 19:06:16
3,64659000,이슈 픽 쌤과 함께,이슈 픽 쌤과 함께 145회,TV 시사/교양,기타,2023-09-08 15:38:22
4,64659000,이슈 픽 쌤과 함께,이슈 픽 쌤과 함께 145회,TV 시사/교양,기타,2023-09-08 14:45:12


In [ ]:
li=content['series_nm'].unique()
for i in li:
  # if i=='':
  #   print('공백존재')
  print(i)

나는자연인이다 
MBC 네트워크 특선 
이슈 픽 쌤과 함께 
휴머니멀 
광복절 의열단 김상옥 
그것이알고싶다 
브런치 톡 
SBS스페셜 
9층 시사국 
방탄 때문에 한글 배웠다 
인간극장 
SDF2022 비호감정치와의이별선언 
알쓸별잡 
과학수사대 스모킹 건 
한문철의 블랙박스 리뷰 
그것이 알고싶다 레전드 50선 
2023 역사저널 그날 
4인용 식탁(채널A) 
꼬리에 꼬리를 무는 그날 이야기 2 
광복절 도쿄공습 프로젝트 윌로우스 
다큐 플렉스 
꼬리에꼬리를무는그날이야기 
실화탐사대 
슈퍼맨이돌아왔다 스페셜 
스타다큐마이웨이 
꼬리에 꼬리를 무는 그날 이야기 1 
다큐인사이트 한우랩소디 
다큐 플러스 
동행 
동물극장 단짝 
글로벌 도네이션쇼 W 
뭐라도 남기리 
전원일기 2021 
다큐인사이트 삼겹살랩소디 
식객 허영만의 백반기행 
특집다큐 백제 무령왕 
NFS 국과수 
기막힌 이야기 실제상황 
PD수첩 
TV동물농장-우린 같이 산다 
기분 좋은 날 
강석우의 종점여행 2 
차이나는 클라스 
오은영 리포트 결혼 지옥 
강력반 X-파일 끝까지 간다 
스트레이트 
휴먼다큐 사노라면 
아날로그 라이프 핸드메이드 
통일전망대 
도망쳐 
다큐인사이트-내일은 아무도 몰라 
당신이 혹하는 사이 4 
광복절 독립운동의 비밀 병기 암호 
tvN STORY 어쩌다 어른 
박원숙의 같이 삽시다 
광복절 조선총독부 최후의 25일 
탐사보도 세븐 
그것이 알고싶다 (2015년)
인간극장 플러스 
우리가 태어난 곳 
UHD로 만나는 TV 문학관 
우리의 얼굴 인간극장 20년 
강석우의 종점여행 
동물의 왕국 
이웃집 찰스 
다큐 인사이트 전쟁과 동맹 
명을 사수하는 사람들 
2023 어린이에게 새 생명을 
주치의 배송 서비스 가족을 부탁해 
추석특집오은영리포트결혼지옥탈출기 
자카르타에서 온 손님 
다큐인사이트 아메리칸 팩토리 
광복절 빼앗긴 날들의 기억 
선을 넘는 녀석들 : 마스터-X 
다큐 인사이트 봉쇄수도원 
한 번 더 리즈시절 
6시 내고향 
청소년 공감콘서트 온드림스쿨 

In [ ]:
li

array(['나는자연인이다 ', 'MBC 네트워크 특선 ', '이슈 픽 쌤과 함께 ', ..., '핑크퐁 그림자극장 ',
       '핑크퐁 3D 율동 동요 ', '브레드이발소 시즌3 Part2 '], dtype=object)

In [ ]:
content['series_nm']

In [ ]:
# content_list = content.drop_duplicates(subset=['super_asset_nm'])[['super_asset_nm', 'ct_cl', 'genre_of_ct_cl']]

In [ ]:
# content_list.head()

In [ ]:
content.to_csv('content_test_mino.csv', index=True, encoding='cp949')

### vod

In [ ]:
vod_temp=vod

In [ ]:
vod=vod_temp

In [ ]:
vod.head()

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,strt_dt
0,64659000,과학수사대 스모킹 건 18회(23/08/02),TV 시사/교양,기타,3120,20230902121124
1,64536000,(HD)그것이알고싶다 1364회(23/08/12),TV 시사/교양,기타,4260,20230930102607
2,66873000,(HD)그것이알고싶다 1366회(23/08/26),TV 시사/교양,기타,4388,20230930042158
3,62421000,과학수사대 스모킹 건 13회(23/06/21),TV 시사/교양,기타,3180,20230902123420
4,66307000,식객 허영만의 백반기행 206회(23/07/07),TV 시사/교양,인물/다큐,244,20230902192639


'기타' 를 가지는 ct_cl 종류

In [ ]:
data=vod[vod['genre_of_ct_cl']=='기타']
data['ct_cl'].unique()

In [ ]:
print('ct_cl의 종류', vod['ct_cl'].unique())
print()
print('ct_cl의 장르 종류',vod['genre_of_ct_cl'].unique())
print()
print('asset_nm unique(회차 포함) 종류 개수',len(vod['asset_nm'].unique()))
print()
print('use_tms unique 개수',len(vod['use_tms'].unique()))
print()
print('vod row 개수', len(vod))

 0   subsr           5042 non-null   int64  <br>
 1   asset_nm        5042 non-null   object 컨텐츠 <br>
 2   ct_cl           5042 non-null   object 컨텐츠 자체 종류<br>
 3   genre_of_ct_cl  5042 non-null   object 해당 컨텐츠의 장르<br>
 4   use_tms         5042 non-null   int64  ???유저 접근 횟수 아니면 유저 아이디<br>
 5   strt_dt         5042 non-null   float64 20230900000000 다 이 값을 가지고 있음

In [ ]:
vod['asset_nm'].unique()

In [ ]:
vod['super_asset_nm'] = vod['asset_nm'].astype(str).apply(lambda x: x if '회' not in x else re.sub(r'\d+\s*회.*', '', x))

In [ ]:
vod['strt_dt'] = vod['strt_dt'].apply(lambda x:pd.to_datetime(str(x),format="%Y%m%d%H%M%S"))

In [ ]:
def is_korean_holiday(date):
    # 주말 여부 확인 (토요일: 5, 일요일: 6)
    if date.weekday() >= 5:
        return 1  # 주말 (토요일 또는 일요일)

    # pytimekr를 사용하여 모든 연도의 공휴일 정보를 가져옴
    holidays = pytimekr.holidays()
    korean_holidays = set()
    for holiday_date in holidays:
        korean_holidays.add(holiday_date)
    if date in korean_holidays:
        return 1  # 공휴일
    else:
        return 0  # 평일
vod['is_holiday'] = vod['strt_dt'].apply(is_korean_holiday)

In [ ]:
vod_list = vod.drop_duplicates(subset=['super_asset_nm'])[['super_asset_nm', 'ct_cl', 'genre_of_ct_cl']]

In [ ]:
vod['date'] = vod['strt_dt'].dt.date
vod['hour'] = vod['strt_dt'].dt.hour
vod['min'] = vod['strt_dt'].dt.minute
vod['weekday'] = vod['strt_dt'].dt.day_name()

In [ ]:
# vod['super_asset_nm'] = vod['super_asset_nm'].str.replace(r'\([^)]*\)', '') 아직 안됨

In [ ]:
vod.head()

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,strt_dt,super_asset_nm
0,64659000,과학수사대 스모킹 건 18회(23/08/02),TV 시사/교양,기타,3120,20230902121124,과학수사대 스모킹 건
1,64536000,(HD)그것이알고싶다 1364회(23/08/12),TV 시사/교양,기타,4260,20230930102607,(HD)그것이알고싶다
2,66873000,(HD)그것이알고싶다 1366회(23/08/26),TV 시사/교양,기타,4388,20230930042158,(HD)그것이알고싶다
3,62421000,과학수사대 스모킹 건 13회(23/06/21),TV 시사/교양,기타,3180,20230902123420,과학수사대 스모킹 건
4,66307000,식객 허영만의 백반기행 206회(23/07/07),TV 시사/교양,인물/다큐,244,20230902192639,식객 허영만의 백반기행


In [ ]:
li=vod['super_asset_nm'].unique()
for i in li:
  if i=='':
    print('공백존재')
  if i=='nan':
    print('nan존재')
  # print(i)

공백존재


In [ ]:
vod2=vod

In [ ]:
import re
li = vod2['super_asset_nm'].unique()
unique_vod = set()
for i in li:
    if isinstance(i, str):
        matches = re.findall(r'\((.*?)\)', i)
        for match in matches:
            unique_vod.add(f'({match})')
for vod in unique_vod:
    print(vod)

In [ ]:
vod_list = vod[vod['ct_cl'] == 'TV 시사/교양']['super_asset_nm'].unique()
vod_list

In [ ]:
grouped = vod.groupby(['weekday', 'hour'])[['ct_cl', 'genre_of_ct_cl', 'super_asset_nm']].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
print(grouped)

In [ ]:
vod['time_group'] = vod['strt_dt'].dt.floor('30T')
result = vod.groupby('time_group').agg({'super_asset_nm': lambda x: x.mode().tolist(), 'ct_cl': pd.Series.mode, 'genre_of_ct_cl': pd.Series.mode})
result.to_csv('result.csv', index=True, encoding='cp949')

### Channel

원래 이름이 Channel_pgm 이었다. 아마 채널 프로그램 데이터 같은데...<br>
해당 셋톱이 어떤 방송사의 어떤 채널프로그램을 언제 봤는지, 그리고 3번 컬럼은..?

In [ ]:
channel.head()

0번 컬럼은 subsr<br>
1번 컬럼은 채널<br>
2번 컬럼은 시간인데 정확히 어떤 시간인지 모름<br>
3번도 미정<br>
4번은 프로그램 이름<br>

In [ ]:
channel.columns = ['subsr', 'channel', 'strt_dt', 'use_tms', 'asset_nm'] # 컬럼이름 변경

In [ ]:
print('channel 종류 개수', len(channel['channel'].unique()))
print()
print('subsr 개수',len(channel['subsr'].unique()))
print()
print('asset_nm unique(회차 포함) 종류 개수',len(channel['asset_nm'].unique()))
print()
print('vod row 개수', len(channel))

### 셋톱박스 사용자 한 명을 확인해보자

In [ ]:
import pandas as pd
if 'subsr' in channel.columns and 'subsr' in vod.columns and 'subsr' in content.columns:
    subsr_channel = channel['subsr'].tolist()
    subsr_vod = vod['subsr'].tolist()
    subsr_content = content['subsr'].tolist()
    subsr_list = list(set(subsr_channel) & set(subsr_vod) & set(subsr_content))
else:
    print("설마 공통된 subsr이 없겠어?")

In [ ]:
subsr_list

NameError: ignored

각 데이터프레임이 의미하는 바가 조금씩 다른가보다.

In [ ]:
# 60224000
print("content에서 subsr이 60224000인 데이터 개수", len(content[content['subsr']==60224000]))
print("vod에서 subsr이 60224000인 데이터 개수",len(vod[vod['subsr']==60224000]))
print("channelt에서 subsr이 60224000인 데이터 개수",len(channel[channel['subsr']==60224000]))

각 데이터 프레임에서 시간 데이터를 뽑아왔다. <br>
시간은 2023 / 09 / 01 부터 2023 / 09 / 30 까지 <br>
각 데이터 프레임마다 또 차이가 존재한다는 것을 알아두자

In [ ]:
print("content에서 subsr 60224000인 데이터의 시간 확인해보기")
print(f"max: {max(content[content['subsr'] == 60224000]['log_dt'])}")
print(f"min: {min(content[content['subsr'] == 60224000]['log_dt'])}")
print()
print("vod에서 subsr 60224000인 데이터의 시간 확인해보기")
print(f"max: {max(vod[vod['subsr'] == 60224000]['strt_dt'])}")
print(f"min: {min(vod[vod['subsr'] == 60224000]['strt_dt'])}")
print()
print("channel에서 subsr 60224000인 데이터의 시간 확인해보기")
print(f"max: {max(channel[channel['subsr'] == 60224000]['strt_dt'])}")
print(f"min: {min(channel[channel['subsr'] == 60224000]['strt_dt'])}")

결국 알아낸 것은, 우리가 얻은 데이터는 2023 년 9월 데이터만을 받은 것이다.<br>
이걸로 웹 페이지를 구축할 때, 추천 모델을 만들어야 한다.


그냥 이렇게 보니까 이해가 안가네 use_tms랑 strt_dt가

In [ ]:
channel[channel['subsr']==60224000][channel['asset_nm'].str.contains('순정복서')]

In [ ]:
channel[(channel['subsr'] == 60224000) & (channel['strt_dt'] // 1000000 == 20230911)].sort_values(by='strt_dt')

In [ ]:
channel[(channel['subsr'] == 60224000) & (channel['strt_dt'] // 1000000 == 20230910)].sort_values(by='strt_dt')

In [ ]:
vod[(vod['subsr'] == 60224000) & (vod['strt_dt'] // 1000000 == 20230910)].sort_values(by='strt_dt')

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,strt_dt
2509,60224000,진짜가 나타났다! 48회(23/09/03),TV드라마,기타,101,20230910031719
2508,60224000,진짜가 나타났다! 48회(23/09/03),TV드라마,기타,1824,20230910044251
2561,60224000,진짜가 나타났다! 48회(23/09/03),TV드라마,기타,173,20230910051523
2519,60224000,진짜가 나타났다! 48회(23/09/03),TV드라마,기타,1992,20230910052124
2541,60224000,진짜가 나타났다! 49회(23/09/09),TV드라마,기타,4200,20230910055439


In [ ]:
content[(content['subsr'] == 60224000) & (content['log_dt'] // 1000000 == 20230910)].sort_values(by='log_dt')

,subsr,series_nm,super_asset_nm,ct_cl,genre_of_ct_cl,log_dt
5557,60224000,연인 파트1-리와인드 필름,연인 파트1-리와인드 필름 1회,TV드라마,기타,20230910031702
3158,60224000,진짜가 나타났다!,진짜가 나타났다! 49회,TV드라마,기타,20230910051419
5710,60224000,진짜가 나타났다!,진짜가 나타났다! 48회,TV드라마,기타,20230910051819
5556,60224000,연인 파트1-리와인드 필름,연인 파트1-리와인드 필름 1회,TV드라마,기타,20230910052035
3156,60224000,진짜가 나타났다!,진짜가 나타났다! 49회,TV드라마,기타,20230910071013


In [ ]:
stats = channel['use_tms'].describe()
print(stats)

In [ ]:
plt.hist(channel['use_tms'], bins=20, color='skyblue')
plt.title('Histogram of use_tms')
plt.xlabel('use_tms')
plt.ylabel('Frequency')
plt.show()

In [ ]:
stats = vod['use_tms'].describe()
print(stats)

In [ ]:
plt.hist(vod['use_tms'], bins=20, color='skyblue')
plt.title('Histogram of use_tms')
plt.xlabel('use_tms')
plt.ylabel('Frequency')
plt.show()

In [ ]:
print(len(vod[vod['use_tms']==0]))
print(len(vod))

**<h1>집가고싶다</h1>**

## Sweetviz

In [ ]:
advert_report = sv.analyze(content)
#display the report
advert_report.show_html('./sweetviz_Advertising_content.html')

In [ ]:
advert_report = sv.analyze(vod)
#display the report
advert_report.show_html('./sweetviz_Advertising_vod.html')

In [ ]:
advert_report = sv.analyze(channel)
#display the report
advert_report.show_html('./sweetviz_Advertising_channel.html')

                                             |          | [  0%]   00:00 -> (? left)

Report ./sweetviz_Advertising_channel.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


## 작업